<a href="https://colab.research.google.com/github/jadinsstewart/Congressional-Analytics/blob/main/Presentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>